In [94]:
#Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import re
from collections import defaultdict

In [95]:
#Use one ./ if running from run_notebooks.py
#Use two ../if running directly from Jupyter Notebooks
#file_beg = '../NHANES-Downloader/data/csv_data/'

file_beg = '../NHANES-Downloader/data/csv_data/'

In [96]:
#Import all the files
files1 = glob.glob(file_beg+'1999-2000/*/*.csv')
files2 = glob.glob(file_beg+'2001-2002/*/*.csv')
files3 = glob.glob(file_beg+'2003-2004/*/*.csv')
files4 = glob.glob(file_beg+'2005-2006/*/*.csv')
files5 = glob.glob(file_beg+'2007-2008/*/*.csv')
files6 = glob.glob(file_beg+'2009-2010/*/*.csv')
files7 = glob.glob(file_beg+'2011-2012/*/*.csv')
files8 = glob.glob(file_beg+'2013-2014/*/*.csv')
files9 = glob.glob(file_beg+'2015-2016/*/*.csv')
files10 = glob.glob(file_beg+'2017-2018/*/*.csv')

In [97]:
#Add files into a list
file_list = [files1, files2, files3, files4, files5, 
          files6, files7, files8, files9, files10]

In [98]:
#Add files into a list of sorted and dictionaries
file_list_dict = []
for x in file_list:
    x.sort()
    file_list_dict.append(dict(enumerate(x)))

In [99]:
demo_indx = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [100]:
dfs = defaultdict(int)
for i in range(0,len(demo_indx)):
    dfs[i] = pd.read_csv(file_list_dict[i][demo_indx[i]])

## Important Variables

In [101]:
#EDIT: Removed INDHHINC from each after DMDHHSIZ and before INDFMINC
#EDIT: Removed INDHHIN2 from each after DMDHHSIZ and before INDFMIN2
#EDIT: Removed DMDAGE before DMDREDU before WT variable

#1999-2002
var_names = ["SEQN", "RIAGENDR", "RIDAGEYR", "RIDRETH1", "DMDBORN", "DMDCITZN", "DMDHHSIZ", "INDFMINC", "DMDHREDU", "WTINT4YR", "WTMEC4YR"]

#2003-2006
#RIDRETH1 is the only race/ethnicity variable in the 2005-2006 Demographics data file. The Demographics files that were released for NHANES 1999-2004 had a variable called ‘RIDRETH2’ that provided an analytic link to the NHANES III race/ethnicity variable.
var_names1 = ["SEQN", "RIAGENDR", "RIDAGEYR", "RIDRETH1", "DMDBORN", "DMDCITZN", "DMDHHSIZ", "INDFMINC", "DMDHREDU", "WTINT2YR", "WTMEC2YR"]

#2007-2010
#DMDHRBRN becomes DMDBORN2
#Recategorize DMDHBRN
#INDFMINC becomes INDFMIN2 - Recategorize 14 and 15 to 11 to match 1999-2006
#INDHHINC becomes INDHHIN2 - Recategorize 14 and 15 to 11 to match 1999-2006
var_names2 = ["SEQN", "RIAGENDR", "RIDAGEYR", "RIDRETH1", "DMDBORN2", "DMDCITZN", "DMDHHSIZ", "INDFMIN2", "DMDHREDU", "WTINT2YR", "WTMEC2YR"]

#2011-2016
#DMDHRBRN4 becomes 2 categories
#Recategorize all previous
var_names3 = ["SEQN", "RIAGENDR", "RIDAGEYR", "RIDRETH1", "DMDBORN4", "DMDCITZN", "DMDHHSIZ", "INDFMIN2", "DMDHREDU", "WTINT2YR", "WTMEC2YR"]

#2017-2018
#DMDHRBRN4 becomes 2 categories
var_names4 = ["SEQN", "RIAGENDR", "RIDAGEYR", "RIDRETH1", "DMDBORN4", "DMDCITZN", "DMDHHSIZ", "INDFMIN2", "DMDHREDZ", "WTINT2YR", "WTMEC2YR"]

descr = ['SEQN', 'Gender', 'Age at Screening (Years)', 'Race/Hispanic Origin', 'Country of Birth', 'Citizenship Status','Total Number of People in the Household','Annual Family Income','Education Level','Two Year Int. Weight','Two Year MEC Weight']

In [102]:
#To display all columns in Jupyter Notebooks
pd.set_option('display.max_columns', 500)

## Functions: Recategorize values, Count Values, Drop Rows

In [103]:
#Recategorize function
def recategorize(df, name, replace_dict):
    df[name].replace(
    to_replace=replace_dict,
    inplace=True
)
    
#Count values function
def count_vals(df, name):
    df_count = df.groupby(name)['SEQN'].nunique()
    print(df_count,"\n\n","NaN: ", df[name].isnull().sum())
    
#Drop rows that include certain values
def drop_rows(df, name, val_list):
    df.drop(df[df[name].isin(val_list)].index, inplace=True)

## Make a copy of the dataframes

In [104]:
df0 = dfs[0][var_names].copy() #1999-2000
df1 = dfs[1][var_names].copy() #2001-2002
df2 = dfs[2][var_names1].copy() #2003-2004
df3 = dfs[3][var_names1].copy() #2005-2006
df4 = dfs[4][var_names2].copy() #2007-2008
df5 = dfs[5][var_names2].copy() #2009-2010
df6 = dfs[6][var_names3].copy() #2011-2012
df7 = dfs[7][var_names3].copy() #2013-2014
df8 = dfs[8][var_names3].copy() #2015-2016
df9 = dfs[9][var_names4].copy() #2017-2018

## Weight for 18-years: 1999 - 2016

In [105]:
#Create column for weight
def reweight(df, col_name, year_wt, weight):
    df[col_name] = df[year_wt]*weight

In [106]:
reweight(df0, 'MEC18YR', 'WTMEC4YR', 2/9)
reweight(df1, 'MEC18YR', 'WTMEC4YR', 2/9)
reweight(df2, 'MEC18YR', 'WTMEC2YR', 1/9)
reweight(df3, 'MEC18YR', 'WTMEC2YR', 1/9)
reweight(df4, 'MEC18YR', 'WTMEC2YR', 1/9)
reweight(df5, 'MEC18YR', 'WTMEC2YR', 1/9)
reweight(df6, 'MEC18YR', 'WTMEC2YR', 1/9)
reweight(df7, 'MEC18YR', 'WTMEC2YR', 1/9)
reweight(df8, 'MEC18YR', 'WTMEC2YR', 1/9)
reweight(df9, 'MEC18YR', 'WTMEC2YR', 1/9)

## Remove 2-yr and 4-yr weights

In [107]:
#Drop columns
def drop_cols(df, cols):
    df.drop(cols, axis=1, inplace=True)

In [108]:
drop_cols(df0,['WTINT4YR','WTMEC4YR'])
drop_cols(df1,['WTINT4YR','WTMEC4YR'])

In [109]:
drop_cols(df2,['WTINT2YR','WTMEC2YR'])
drop_cols(df3,['WTINT2YR','WTMEC2YR'])
drop_cols(df4,['WTINT2YR','WTMEC2YR'])
drop_cols(df5,['WTINT2YR','WTMEC2YR'])
drop_cols(df6,['WTINT2YR','WTMEC2YR'])
drop_cols(df7,['WTINT2YR','WTMEC2YR'])
drop_cols(df8,['WTINT2YR','WTMEC2YR'])
drop_cols(df9,['WTINT2YR','WTMEC2YR'])

In [110]:
df0[:5]

,SEQN,RIAGENDR,RIDAGEYR,RIDRETH1,DMDBORN,DMDCITZN,DMDHHSIZ,INDFMINC,DMDHREDU,MEC18YR
0,1.0,2.0,2.0,4.0,1.0,1.0,3.0,3.0,3.0,990.268132
1,2.0,1.0,77.0,3.0,1.0,1.0,1.0,8.0,5.0,3408.044382
2,3.0,2.0,10.0,3.0,3.0,2.0,4.0,6.0,4.0,4724.103694
3,4.0,1.0,1.0,4.0,1.0,1.0,7.0,3.0,3.0,1013.864237
4,5.0,1.0,49.0,3.0,1.0,1.0,3.0,11.0,4.0,10219.103963


## Recategorize values

In [111]:
#Recategorize DMDBORN to 2 categories
recategorize(df0, 'DMDBORN', {3:2, 7:77, 9:99}) #1999-2000
recategorize(df1, 'DMDBORN', {3:2, 7:77, 9:99}) #2001-2002
recategorize(df2, 'DMDBORN', {3:2, 7:77, 9:99}) #2003-2004
recategorize(df3, 'DMDBORN', {3:2, 7:77, 9:99}) #2005-2006


#Recategorize DMDBORN2 to 2 categories
#Recategorize INHHIN2 to INDHHINC
#Recategorize INDFMIN2 to INDFMINC

#2007-2008
recategorize(df4, 'DMDBORN2', {4:2, 5:2, 7:77, 9:99})
#recategorize(df4, 'INDHHIN2', {14:11, 15:11})
recategorize(df4, 'INDFMIN2', {14:11, 15:11})

#2009-2010
recategorize(df5, 'DMDBORN2', {4:2, 5:2, 7:77, 9:99})
#recategorize(df5, 'INDHHIN2', {14:11, 15:11})
recategorize(df5, 'INDFMIN2', {14:11, 15:11})

#2011-2012
#recategorize(df6, 'INDHHIN2', {14:11, 15:11})
recategorize(df6, 'INDFMIN2', {14:11, 15:11})

#2013-2014
#recategorize(df7, 'INDHHIN2', {14:11, 15:11})
recategorize(df7, 'INDFMIN2', {14:11, 15:11})

#2015-2018
#recategorize(df8, 'INDHHIN2', {14:11, 15:11})
recategorize(df8, 'INDFMIN2', {14:11, 15:11})
recategorize(df9, 'INDFMIN2', {14:11, 15:11})

## Rename columns 1999 - 2018

In [112]:
#New column names
col_names = ["SEQN", "RIAGENDR", "RIDAGEYR", "RIDRETH1", "DMDBORN4", "DMDCITZN", "DMDHHSIZ", "INDFMINC", "DMDHREDU", "MEC18YR"]

In [113]:
df0.columns = col_names
df1.columns = col_names
df2.columns = col_names
df3.columns = col_names
df4.columns = col_names
df5.columns = col_names
df6.columns = col_names
df7.columns = col_names
df8.columns = col_names
df9.columns = col_names

## Append years 1999 - 2016

In [114]:
years = ["1999-2000","2001-2002","2003-2004","2005-2006","2007-2008", 
        "2009-2010","2011-2012","2013-2014","2015-2016","2017-2018"]

In [115]:
frames = [df0, df1, df2, df3, df4, df5, df6, df7, df8, df9]

In [116]:
#Add years as a column
for i, df in enumerate(frames):
    df["Year"] = years[i]

In [117]:
result_1999_2018 = pd.concat(frames, keys = years)

In [118]:
result_1999_2018_cleaned = result_1999_2018.copy()

In [119]:
#Getting rows by keys
#result_1999_2016.loc[years[0]][:5]

In [120]:
len(result_1999_2018)

101316

## Count values for each column

In [121]:
len(col_names)

10

In [122]:
#RIAGENDR - Gender of the sample person
count_vals(result_1999_2018, col_names[1])

RIAGENDR
1.0    49893
2.0    51423
Name: SEQN, dtype: int64 

 NaN:  0


In [123]:
#RIDAGEYR - Age at Screening Adjudicated
#count_vals(result_1999_2016, col_names[2])

In [124]:
#RIDRETH1 - Race/Ethnicity
count_vals(result_1999_2018, col_names[3])

RIDRETH1
1.0    22449
2.0     8294
3.0    37432
4.0    23644
5.0     9497
Name: SEQN, dtype: int64 

 NaN:  0


In [125]:
#DMDBORN4 - Country of Birth
count_vals(result_1999_2018, col_names[4])

DMDBORN4
1.0     82845
2.0     18414
77.0       31
99.0       10
Name: SEQN, dtype: int64 

 NaN:  16


In [126]:
#DMDCITZN - Citizenship status
count_vals(result_1999_2018, col_names[5])

DMDCITZN
1.0    90534
2.0    10562
7.0      131
9.0       55
Name: SEQN, dtype: int64 

 NaN:  34


In [127]:
#DMDHHSIZ - Total number of people in the Household
count_vals(result_1999_2018, col_names[6])

DMDHHSIZ
1.0     8215
2.0    19129
3.0    17791
4.0    21219
5.0    16336
6.0     8830
7.0     9796
Name: SEQN, dtype: int64 

 NaN:  0


In [128]:
#INDHHINC - Annual Household Income
#count_vals(result_1999_2016, col_names[7])

In [129]:
#INDFMINC - Annual Family Income
count_vals(result_1999_2018, col_names[7])

INDFMINC
1.0      4254
2.0      5651
3.0      8345
4.0      7779
5.0      8288
6.0     11764
7.0      9061
8.0      7327
9.0      5336
10.0     4296
11.0    20019
12.0     2546
13.0     1529
77.0     1826
99.0     1509
Name: SEQN, dtype: int64 

 NaN:  1786


In [130]:
#DMDHRAGE - HH Ref Person Age
#count_vals(result_1999_2016, col_names[9])

In [131]:
#DMDHREDU - HH reference person's education level
count_vals(result_1999_2018, col_names[8])

DMDHREDU
1.0    12725
2.0    20185
3.0    22942
4.0    23868
5.0    17656
7.0       65
9.0      372
Name: SEQN, dtype: int64 

 NaN:  3503


In [132]:
len(result_1999_2018)

101316

In [133]:
result_1999_2018_cleaned[:5]

SEQN  RIAGENDR  RIDAGEYR  RIDRETH1  DMDBORN4  DMDCITZN  DMDHHSIZ  \
1999-2000 0   1.0       2.0       2.0       4.0       1.0       1.0       3.0   
          1   2.0       1.0      77.0       3.0       1.0       1.0       1.0   
          2   3.0       2.0      10.0       3.0       2.0       2.0       4.0   
          3   4.0       1.0       1.0       4.0       1.0       1.0       7.0   
          4   5.0       1.0      49.0       3.0       1.0       1.0       3.0   

             INDFMINC  DMDHREDU       MEC18YR       Year  
1999-2000 0       3.0       3.0    990.268132  1999-2000  
          1       8.0       5.0   3408.044382  1999-2000  
          2       6.0       4.0   4724.103694  1999-2000  
          3       3.0       3.0   1013.864237  1999-2000  
          4      11.0       4.0  10219.103963  1999-2000

## Remove rows with missing values:

In [134]:
drop_rows(result_1999_2018_cleaned, col_names[1], [np.nan])
drop_rows(result_1999_2018_cleaned, col_names[2], [np.nan])
drop_rows(result_1999_2018_cleaned, col_names[3], [np.nan])
drop_rows(result_1999_2018_cleaned, col_names[4], [np.nan, 77, 99])
drop_rows(result_1999_2018_cleaned, col_names[5], [np.nan, 7, 9])
drop_rows(result_1999_2018_cleaned, col_names[6], [np.nan])
drop_rows(result_1999_2018_cleaned, col_names[7], [np.nan, 12, 13, 77, 99])
#drop_rows(result_1999_2018_cleaned, col_names[8], [np.nan, 13, 13, 77, 99]) #Previously INDHHINC
#drop_rows(result_1999_2018_cleaned, col_names[8], [np.nan]) #Previously DMDHREDU
drop_rows(result_1999_2018_cleaned, col_names[8], [np.nan, 7, 9]) 

In [135]:
len(result_1999_2018_cleaned)

89367

In [136]:
result_1999_2018_cleaned[:5]

SEQN  RIAGENDR  RIDAGEYR  RIDRETH1  DMDBORN4  DMDCITZN  DMDHHSIZ  \
1999-2000 0   1.0       2.0       2.0       4.0       1.0       1.0       3.0   
          1   2.0       1.0      77.0       3.0       1.0       1.0       1.0   
          2   3.0       2.0      10.0       3.0       2.0       2.0       4.0   
          3   4.0       1.0       1.0       4.0       1.0       1.0       7.0   
          4   5.0       1.0      49.0       3.0       1.0       1.0       3.0   

             INDFMINC  DMDHREDU       MEC18YR       Year  
1999-2000 0       3.0       3.0    990.268132  1999-2000  
          1       8.0       5.0   3408.044382  1999-2000  
          2       6.0       4.0   4724.103694  1999-2000  
          3       3.0       3.0   1013.864237  1999-2000  
          4      11.0       4.0  10219.103963  1999-2000

## See if missing values have been correctly removed:

In [137]:
count_vals(result_1999_2018_cleaned, col_names[1])

RIAGENDR
1.0    44195
2.0    45172
Name: SEQN, dtype: int64 

 NaN:  0


In [138]:
count_vals(result_1999_2018_cleaned, col_names[2])

RIDAGEYR
5.397605e-79    4040
1.000000e+00    2657
2.000000e+00    2715
3.000000e+00    1846
4.000000e+00    1958
                ... 
8.100000e+01     232
8.200000e+01     186
8.300000e+01     136
8.400000e+01     154
8.500000e+01     649
Name: SEQN, Length: 86, dtype: int64 

 NaN:  0


In [139]:
count_vals(result_1999_2018_cleaned, col_names[3])

RIDRETH1
1.0    19144
2.0     6962
3.0    34245
4.0    20873
5.0     8143
Name: SEQN, dtype: int64 

 NaN:  0


In [140]:
count_vals(result_1999_2018_cleaned, col_names[4])

DMDBORN4
1.0    74118
2.0    15249
Name: SEQN, dtype: int64 

 NaN:  0


In [141]:
count_vals(result_1999_2018_cleaned, col_names[5])

DMDCITZN
1.0    80678
2.0     8689
Name: SEQN, dtype: int64 

 NaN:  0


In [142]:
count_vals(result_1999_2018_cleaned, col_names[6])

DMDHHSIZ
1.0     7537
2.0    17102
3.0    15906
4.0    18824
5.0    14447
6.0     7644
7.0     7907
Name: SEQN, dtype: int64 

 NaN:  0


In [143]:
count_vals(result_1999_2018_cleaned, col_names[7])

INDFMINC
1.0      4018
2.0      5468
3.0      8093
4.0      7529
5.0      8017
6.0     11496
7.0      8843
8.0      7099
9.0      5214
10.0     4189
11.0    19401
Name: SEQN, dtype: int64 

 NaN:  0


In [144]:
count_vals(result_1999_2018_cleaned, col_names[8])

DMDHREDU
1.0    11061
2.0    18276
3.0    21056
4.0    22351
5.0    16623
Name: SEQN, dtype: int64 

 NaN:  0


In [145]:
before = len(result_1999_2018)
before

101316

In [146]:
after = len(result_1999_2018_cleaned)
after

89367

In [147]:
(before-after)/after

0.13370707308066737

## MongoDB Insertion

In [148]:
#Import MongoClient
from pymongo import MongoClient

In [149]:
#Create a MongoClient to run the MongoDB instance
client = MongoClient("localhost", 27017)

In [150]:
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [151]:
#Creating a database
db = client['NHANES']
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'NHANES')

In [152]:
db.list_collection_names()

['hiq',
 'alq',
 'drxtot',
 'smqfam',
 'smq',
 'bpq',
 'bmx',
 'whq',
 'huq',
 'bpx',
 'mcq_h',
 'tchol',
 'paq',
 'demo_p',
 'mcq_a',
 'mcq_b',
 'rdq',
 'demo',
 'diq',
 'mcq_c']

In [153]:
#Creating a collection
demo = db.demo

In [154]:
#If collections exist, then drop
if 'demo' in db.list_collection_names():
    demo.drop()
    db.list_collection_names()

## Inputting into DB

In [155]:
result_1999_2018_cleaned.rename(columns= {'SEQN':'_id'}, inplace=True)

In [156]:
result_1999_2018_cleaned[:3]

_id  RIAGENDR  RIDAGEYR  RIDRETH1  DMDBORN4  DMDCITZN  DMDHHSIZ  \
1999-2000 0  1.0       2.0       2.0       4.0       1.0       1.0       3.0   
          1  2.0       1.0      77.0       3.0       1.0       1.0       1.0   
          2  3.0       2.0      10.0       3.0       2.0       2.0       4.0   

             INDFMINC  DMDHREDU      MEC18YR       Year  
1999-2000 0       3.0       3.0   990.268132  1999-2000  
          1       8.0       5.0  3408.044382  1999-2000  
          2       6.0       4.0  4724.103694  1999-2000

In [157]:
demo_dict = result_1999_2018_cleaned.to_dict(orient='records')

In [158]:
demo_dict[0]

{'_id': 1.0,
 'RIAGENDR': 2.0,
 'RIDAGEYR': 2.0,
 'RIDRETH1': 4.0,
 'DMDBORN4': 1.0,
 'DMDCITZN': 1.0,
 'DMDHHSIZ': 3.0,
 'INDFMINC': 3.0,
 'DMDHREDU': 3.0,
 'MEC18YR': 990.268132,
 'Year': '1999-2000'}

In [159]:
#Insert collection
demo.insert_many(demo_dict)

In [160]:
db.list_collection_names()

['hiq',
 'alq',
 'drxtot',
 'smqfam',
 'smq',
 'bpq',
 'bmx',
 'whq',
 'demo',
 'huq',
 'bpx',
 'mcq_h',
 'tchol',
 'paq',
 'demo_p',
 'mcq_a',
 'mcq_b',
 'rdq',
 'diq',
 'mcq_c']

In [161]:
#for d in demo.find():
    #print(d)